## Introduction

Misdiagnosis of the many diseases impacting agricultural crops can lead to misuse of chemicals and more outbreaks with significant economic loss and environmental impacts. Current disease diagnosis based on human is time-consuming and expensive, and although computer-vision based models have the promise to increase efficiency.

## Specific Objectives

Objectives of ‘Cassava Leaf Disease Classification Challenge’ is to train a model using images of training dataset to 1) Accurately classify a given image from testing dataset into different diseased category or a healthy leaf;  2) Address depth perception—angle, light, shade, physiological age of the leaf because as an added challenge, effective solutions for farmers must perform well under significant constraints, since African farmers may only have access to mobile-quality cameras with low-bandwidth; 

## Data

### Data Description
Given a photo of an Cassava leaf, can you accurately assess its health? This competition will challenge you to classify each cassava image into four disease categories or a fifth category indicating a healthy leaf. 

### Files

***train.csv***

``image_id``: The id defined for each training image.

``label``: Number defined for each disease.

***test.csv***

``image_id``: The id defined for each testing image.

-------------------------------------------------------

***train images***

Training images with name as image id and in jpg format.

***test images***

Testing images with name as image id and in jpg format.

-------------------------------------------------------

***label_num_to_disease_map.json***

Disease name encoded as label number in train.csv 

## Importing required fastai modules and packages

package fastai.vision provides infinite number of functionalities to deal with computer vision problems like image classification, image segmentation etc...

In [ ]:
!pip install ../input/timm-library/timm-0.3.1-py3-none-any.whl

In [ ]:
# Data Processing
import numpy as np 
import pandas as pd

# Model
from fastai.vision.all import *
import torchvision.models as models 

# utils
import json

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')
os.listdir(path)

In [ ]:
# Lets take a look at the CSV.
train = pd.read_csv(path/"train.csv")
train.head()

Okay let's check how many images are available in the training dataset:

In [ ]:
len(train)

Hopefully, we can develop a highly-predictive, robust, and generalizable model with this dataset.

Let's check the distribution of the different classes:

In [ ]:
train['label'].hist(figsize = (10, 5))

From above plot, we have 5 labels (4 diseases and 1 healthy):

1. Cassava Bacterial Blight (CBB)
2. Cassava Brown Streak Disease (CBSD)
3. Cassava Green Mottle (CGM)
4. Cassava Mosaic Disease (CMD)
5. Healthy

In this case label 3, Cassava Mosaic Disease (CMD) is the most common label. This imbalance may have to be addressed with a weighted loss function or oversampling.

Let us define classes as follows

In [ ]:
f = open (path/'label_num_to_disease_map.json', "r") 

data = json.loads(f.read()) 
print(data)
f.close()

### Add complete path column to image_id of train.csv

In [ ]:
train['path'] = train['image_id'].map(lambda x:path/'train_images'/x)
train = train.drop(columns=['image_id'])
train = train.sample(frac=1).reset_index(drop=True) #shuffle dataframe
train.head(10)

### DataBlock

First let's define item and batch transforms and we'll come up with some basic data augmentations.

Our ``item_tfms`` should ensure everything is ready to go into a batch, so we will use Resize.

Our ``batch_tfms`` should apply any extra augmentations we may want. We'll use RandomResizedCropGPU, aug_transforms, and apply our Normalize:

We will normalize our data based on ImageNet, since that is what our pretrained model was trained with

In [ ]:
item_tfms = RandomResizedCrop(460, min_scale=0.75, ratio=(1.,1.))
batch_tfms = [*aug_transforms(size=224, max_warp=0), Normalize.from_stats(*imagenet_stats)]
bs=32

In [ ]:
dls = ImageDataLoaders.from_df(train, #pass in train DataFrame
                               valid_pct=0.2, #80-20 train-validation random split
                               seed=999, #seed
                               label_col=0, #label is in the first column of the DataFrame
                               fn_col=1, #filename/path is in the second column of the DataFrame
                               bs=bs, #pass in batch size
                               item_tfms=item_tfms, #pass in item_tfms
                               batch_tfms=batch_tfms)

In [ ]:
dls.show_batch()

### Model 

In this competition we are going to use ``timm`` library by Ross Wightman. Let's add pretrained ``EfficientNet-B3`` model's weight. 

as we are going to use timm  we won't be using ``cnn_learner`` insted we will be using ``timm_learner`` because ``cnn_learner`` doesn't support the models from ``timm`` library. We could use ``resnet/18/34/50`` pretrained model, and we'll be using the ``accuracy metric`` as this is how this competition will grade our results with. We can also use ``mixed precision`` very easily, along with ``Label Smoothing``

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')

!cp '../input/../input/efficientnet-b3/tf_efficientnet_b3_ns-9d44bf68.pth' '/root/.cache/torch/hub/checkpoints/tf_efficientnet_b3_ns-9d44bf68.pth'

In [ ]:
from timm import create_model
from fastai.vision.learner import _update_first_layer

def create_timm_body(arch:str, pretrained=True, cut=None, n_in=3):
    "Creates a body from any model in the `timm` library."
    model = create_model(arch, pretrained=pretrained, num_classes=0, global_pool='')
    _update_first_layer(model, n_in, pretrained)
    if cut is None:
        ll = list(enumerate(model.children()))
        cut = next(i for i,o in reversed(ll) if has_pool_type(o))
    if isinstance(cut, int): return nn.Sequential(*list(model.children())[:cut])
    elif callable(cut): return cut(model)
    else: raise NamedError("cut must be either integer or function")

In [ ]:
def create_timm_model(arch:str, n_out, cut=None, pretrained=True, n_in=3, init=nn.init.kaiming_normal_, custom_head=None,
                     concat_pool=True, **kwargs):
    "Create custom architecture using `arch`, `n_in` and `n_out` from the `timm` library"
    body = create_timm_body(arch, pretrained, None, n_in)
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children())) * (2 if concat_pool else 1)
        head = create_head(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    return model

In [ ]:
def timm_learner(dls, arch:str, loss_func=None, pretrained=True, cut=None, splitter=None,
                y_range=None, config=None, n_out=None, normalize=True, **kwargs):
    "Build a convnet style learner from `dls` and `arch` using the `timm` library"
    if config is None: config = {}
    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    if y_range is None and 'y_range' in config: y_range = config.pop('y_range')
    model = create_timm_model(arch, n_out, default_split, pretrained, y_range=y_range, **config)
    learn = Learner(dls, model, loss_func=loss_func, splitter=default_split, **kwargs)
    if pretrained: learn.freeze()
    return learn

In [ ]:
learner = timm_learner(dls, 
                    'tf_efficientnet_b3_ns', 
                     opt_func = ranger,
                     loss_func=LabelSmoothingCrossEntropy(),
                     metrics = [accuracy]).to_native_fp16()

### Finding the learning rate

Fastai provide functionality to find optimized learning rate. In order to find the optimized learning rate, the method ``lr_find()`` can be used. The method plot() defined on recorder can be used to plot a line plot between Loss vs. learning Rate. Fastai provide the suggestion also for optimize learning rate

In [ ]:
learner.model_dir = '/kaggle/working/models'

In [ ]:
learner.lr_find()

For fastai, the best way to train a model is to train the frozen pretrained model for a single epoch then train the whole pretrained model for several epochs.

As shown above, the optimal learning rate for training the frozen model is where the loss is decreasing quickly around ~1e-1. To be safe, we will use high weight decay to help prevent overfitting. We will also use another common state-of-the-art training technique: mixup.

In [ ]:
learner.freeze()
learner.fit_flat_cos(1,1e-1, wd=0.5, cbs=[MixUp()])

In [ ]:
learner.save('stage-1')

In [ ]:
learner = learner.load('stage-1')

In [ ]:
learner.unfreeze()
learner.lr_find()

In [ ]:
learner.unfreeze()
learner.fit_flat_cos(10,5e-3,pct_start=0,cbs=[MixUp()])

In [ ]:
learner.recorder.plot_loss()

put the model back to fp32, and now we can export the model if we want to use later (i.e. for an inference).

In [ ]:
learner = learner.to_native_fp32()

In [ ]:
learner.save('stage-2')

In [ ]:
learner.export()

fastai also comes with some additional utilities like checking the confusion matrix:

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_confusion_matrix()

### Inference
It's very simple to perform inference with fastai. The dls.test_dl function allows you to create test dataloader using the same pipeline defined earlier.

In [ ]:
sample = pd.read_csv(path/'sample_submission.csv')
sample

In [ ]:
_sample = sample.copy()
_sample['path'] = _sample['image_id'].map(lambda x:path/'test_images'/x)
_sample = _sample.drop(columns=['image_id'])
test_dl = dls.test_dl(_sample)

In [ ]:
test_dl.show_batch()

Now let's pass the dataloader to the model and get predictions. We will use a common inference technique known as test-time augmentation (average predictions when passing in various augmented versions of the test image). This is also implemented in fastai. Let's do 8x TTA

In [ ]:
preds, _ = learner.tta(dl=test_dl, n=8, beta=0)

Let's make a submission with these predictions!

In [ ]:
sample['label'] = preds.argmax(dim=-1).numpy()

In [ ]:
sample.to_csv('submission.csv',index=False)

### And Done, let's submit our submission.csv file 